**A continuación definimos la clase de nodo y de árbol de merkle**
Para la construcción del árbol se toman las hojas y recursivamente se va construyendo hacia arriba piso por piso, luego recursivamente desde la raíz hacia las hojas se añade la relacion de padre, finalmente la funcion que entrega la prueba revisa primero si dicho valor es parte de la hojas y en el caso de que sea recorre el árbol hacia arriba según la relacion de padre hasta llegar a la raíz para determinar cual es la informacion necesaria para la prueba.


In [0]:
class Node:
    def __init__(self, left, right, value, ):
        self.parent = None
        self.left = left
        self.right = right
        self.value = value

    def __repr__(self):
        return f" mi hash es: {self.value}"


class MerkleTree:
    def __init__(self, values, hash_func):
        self.values = values
        self.hash_func = hash_func
        self.root = None
        self.leaves = []
        self.build_tree(values)

    def build_tree(self, values) -> None:
        self.leaves = [Node(None, None, self.hash_func(e)) for e in values]
        self.root = self.build_tree_rec(self.leaves)
        self.emparentar(self.root)

    def get_root(self):
        return self.root.value

    def build_tree_rec(self, nodes):
        if len(nodes) == 2:
            return Node(nodes[0], nodes[1], self.hash_func(nodes[0].value + nodes[1].value))

        if len(nodes) % 2 == 1:
            nodes.append(nodes[-1:][0])  # duplicate last elem if odd number of elements

        new_level = []
        for i in range(len(nodes) // 2):
            node_left = nodes[2 * i]
            node_right = nodes[2 * i + 1]
            new_level.append(Node(node_left, node_right, self.hash_func(node_left.value + node_right.value)))
        return self.build_tree_rec(new_level)

    def printTree(self):
        self.printTreeRec(self.root)

    def printTreeRec(self, node):
        if node != None:
            print(node.value)
            self.printTreeRec(node.left)
            self.printTreeRec(node.right)

    def get_proof_for(self, item):
        if item not in self.values:
            return None

        idx = self.values.index(item)
        leaf = self.leaves[idx]
        proof = list()
        self.get_proof(leaf, proof)
        """
        Returns :
        result : None if the item is not part of the leafs of the tree
        A list with the necessary info to prove that the
        item is part of the leafs of the tree
        """
        return proof

    def get_proof(self, node, proof):
        if node.parent is None:
            return
        if node.parent.left == node:
            proof.append((node.parent.right.value, "d"))
        else:
            proof.append((node.parent.left.value, "i"))
        self.get_proof(node.parent, proof)

    def emparentar(self, node):
        if node is None:
            return
        if node.left:
            node.left.parent = node
            node.right.parent = node
        self.emparentar(node.right)
        self.emparentar(node.left)

    def find_leaf(self, node, hash):
        if node is None:
            return False
        if node.value == hash:
            return True
        res_left = self.find_leaf(node.left, hash)
        if res_left:
            return True
        res_right = self.find_leaf(node.right, hash)
        return res_right

**Definimos la función de verificación**
esta función va calculando el hash final elemento a elemento, decidiendo si concatena las palabras a la izquierda o derecha según corresponda

In [0]:
def verify(root, item, proof, hash_func):
    first = hash_func(item)
    proof = proof
    valor = first
    for tuple in proof:
        if tuple[1] == "d":
            valor = hash_func(valor + tuple[0])
        elif tuple[1] == "i":
            valor = hash_func(tuple[0] + valor)
    return root == valor